In [33]:
#import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.compose import ColumnTransformer,TransformedTargetRegressor
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler,OrdinalEncoder,PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor,StackingRegressor
from sklearn.metrics import r2_score,mean_absolute_error

In [2]:
df=pd.read_csv('/content/final_df (4).csv')
df.head()

,person_age,person_ratings,latitude,longitude,delivery_location_latitude,delivery_location_longitude,weather_condition,traffic_density,vehicle_condition,order_type,...,city_name,day,month,day_of_week,is_weekend,pickup_time_in_minutes,order_time_hr,time_slot,distance,distance_type
0,37.0,4.9,22.745049,75.892471,22.765049,75.912471,sunny,High,2,Snack,...,INDO,19,3,Saturday,1,15.0,11.0,Morning,3.025149,short
1,34.0,4.5,12.913041,77.683237,13.043041,77.813237,stormy,Jam,2,Snack,...,BANG,25,3,Friday,0,5.0,19.0,Evening,20.183530,very long
2,23.0,4.4,12.914264,77.678400,12.924264,77.688400,sandstorms,Low,0,Drinks,...,BANG,19,3,Saturday,1,15.0,8.0,Morning,1.552758,short
3,38.0,4.7,11.003669,76.976494,11.053669,77.026494,sunny,Medium,0,Buffet,...,COIMB,5,4,Tuesday,0,10.0,18.0,Evening,7.790401,medium
4,32.0,4.6,12.972793,80.249982,13.012793,80.289982,cloudy,High,1,Snack,...,CHEN,26,3,Saturday,1,15.0,13.0,Afternoon,6.210138,medium


In [3]:
!pip install dagshub mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.0/684.0 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [4]:
import dagshub
dagshub.init(repo_owner='amitkumar981', repo_name='swiggy-delivery-time-prediction', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=96d10896-556e-4152-9d38-43762ee037c1&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=14b351118c9776db237f575c663f4ae29a0438ce0b44fe279a323b44e41e4393




Accessing as amitkumar981

Initialized MLflow to track repo "amitkumar981/swiggy-delivery-time-prediction"

Repository amitkumar981/swiggy-delivery-time-prediction initialized!

In [5]:
import mlflow
mlflow.set_tracking_uri('https://dagshub.com/amitkumar981/swiggy-delivery-time-prediction.mlflow')

In [6]:
mlflow.set_experiment('exp-9 built final esimator')

2025/04/03 01:25:08 INFO mlflow.tracking.fluent: Experiment with name 'exp-9 built final esimator' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/176cb4bc961b459a9f0c9214e06c483f', creation_time=1743643508375, experiment_id='9', last_update_time=1743643508375, lifecycle_stage='active', name='exp-9 built final esimator', tags={}>

In [7]:
#drop missing values from dataset
df.dropna(inplace=True)

In [9]:
#drop unessary columns from dataset
columns_to_drop=['latitude', 'longitude','delivery_location_latitude', 'delivery_location_longitude','order_time_hr']
df.drop(columns=columns_to_drop,axis=1,inplace=True)

In [12]:
#saprate dependent and independent variablr from dataset
x=df.drop(columns='time_taken')
y=df['time_taken']

In [15]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)
x_train.shape,x_test.shape

((30451, 19), (7613, 19))

In [17]:
#numarical columns
num_cols=['person_age','person_ratings','pickup_time_in_minutes','distance']
num_cat_cols=['weather_condition','order_type','vehicle_type','multiple_deliveries','city','festival',
             'city_name','day_of_week','time_slot']
ordinal_cat_cols=['traffic_density','distance_type']

In [18]:
traffic_order=['Low ','Medium ','High ','Jam ']
distance_order=['short','medium','long','very long']

In [21]:
preprocessor=ColumnTransformer(transformers=[
    ('scaling',MinMaxScaler(),num_cols),
    ('OHE',OneHotEncoder(sparse_output=False,drop='first',handle_unknown='ignore'),num_cat_cols),
    ('ordinal_encoder',OrdinalEncoder(categories=[traffic_order,distance_order]),ordinal_cat_cols)
],remainder='passthrough',verbose_feature_names_out=False,n_jobs=-1,force_int_remainder_cols=False)

preprocessor.set_output(transform='pandas')

ColumnTransformer(force_int_remainder_cols=False, n_jobs=-1,
                  remainder='passthrough',
                  transformers=[('scaling', MinMaxScaler(),
                                 ['person_age', 'person_ratings',
                                  'pickup_time_in_minutes', 'distance']),
                                ('OHE',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore',
                                               sparse_output=False),
                                 ['weather_condition', 'order_type',
                                  'vehicle_type', 'multiple_deliveries', 'city',
                                  'festival', 'city_name', 'day_of_week',
                                  'time_slot']),
                                ('ordinal_encoder',
                                 OrdinalEncoder(categories=[['Low ', 'Medium ',
                                                             'High ', 'Jam '],
                                                            ['short', 'medium',
                                                             'long',
                                                             'very long']]),
                                 ['traffic_density', 'distance_type'])],
                  verbose_feature_names_out=False)

In [38]:
#apply power transformer on target column
pt=PowerTransformer()
y_train_pt=pt.fit_transform(y_train.values.reshape(-1,1))
y_test_pt=pt.transform(y_test.values.reshape(-1,1))

In [39]:
x_train_trans=preprocessor.fit_transform(x_train)
x_test_trans=preprocessor.transform(x_test)
x_train_trans

,person_age,person_ratings,pickup_time_in_minutes,distance,weather_condition_fog,weather_condition_sandstorms,weather_condition_stormy,weather_condition_sunny,weather_condition_windy,order_type_Drinks,...,day_of_week_Wednesday,time_slot_Evening,time_slot_Morning,time_slot_Night,traffic_density,distance_type,vehicle_condition,day,month,is_weekend
24939,0.052632,0.80,0.5,0.163629,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,3.0,0.0,1,9,3,0
9439,0.526316,0.96,1.0,0.157036,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,3.0,0.0,2,3,3,0
18409,0.631579,1.00,0.0,0.160160,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,3.0,0.0,2,13,3,1
15267,0.789474,0.96,0.5,0.003145,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,2,11,2,0
43924,0.157895,0.88,0.5,0.084651,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,2,11,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20042,0.368421,0.88,1.0,0.621266,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,2.0,2,2,4,1
7521,0.842105,0.88,0.0,0.788907,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,3.0,3.0,0,12,3,1
13483,0.736842,0.92,0.5,0.154044,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,2,17,3,0
1038,0.368421,0.92,0.5,0.992901,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,3.0,2,16,2,0


In [40]:
lgbm_params={'num_leaves': 80,
 'max_depth': 7,
 'learning_rate': 0.07521051144940308,
 'n_estimators': 350,
 'lambda_l1': 4.994553000952651,
 'lambda_l2': 1.8523641834042492,
 'min_data_in_leaf': 45,
 'feature_fraction': 0.7095710565305883,
 'bagging_fraction': 0.9976539330187573,
 'bagging_freq': 4}

rf_params={'n_estimators': 400,
 'max_depth': 21,
 'min_samples_split': 18,
 'min_samples_leaf': 10,
 'bootstrap': True,
 'max_samples': 0.7634340439115328,
 'min_impurity_decrease': 3.465395379320018e-05}

best_lgbm=LGBMRegressor(**lgbm_params)

best_rf=RandomForestRegressor(**rf_params)

Lr=LinearRegression()


In [41]:
stacking_reg=StackingRegressor(estimators=[('lgbm_reg',best_lgbm),('rf_reg',best_rf)],
                                final_estimator=Lr,cv=5,n_jobs=-1)

In [44]:
Pt=PowerTransformer()
model=TransformedTargetRegressor(regressor=stacking_reg,transformer=pt)
model.fit(x_train_trans,y_train)

TransformedTargetRegressor(regressor=StackingRegressor(cv=5,
                                                       estimators=[('lgbm_reg',
                                                                    LGBMRegressor(bagging_fraction=0.9976539330187573,
                                                                                  bagging_freq=4,
                                                                                  feature_fraction=0.7095710565305883,
                                                                                  lambda_l1=4.994553000952651,
                                                                                  lambda_l2=1.8523641834042492,
                                                                                  learning_rate=0.07521051144940308,
                                                                                  max_depth=7,
                                                                                  min_data_in_leaf=45,
                                                                                  n_estimators=350,
                                                                                  num_leaves=80)),
                                                                   ('rf_reg',
                                                                    RandomForestRegressor(max_depth=21,
                                                                                          max_samples=0.7634340439115328,
                                                                                          min_impurity_decrease=3.465395379320018e-05,
                                                                                          min_samples_leaf=10,
                                                                                          min_samples_split=18,
                                                                                          n_estimators=400))],
                                                       final_estimator=LinearRegression(),
                                                       n_jobs=-1),
                           transformer=PowerTransformer())

In [46]:
y_pred_train=model.predict(x_train_trans)
y_pred_test=model.predict(x_test_trans)


[LightGBM] [Warning] min_data_in_leaf is set=45, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=45
[LightGBM] [Warning] feature_fraction is set=0.7095710565305883, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7095710565305883
[LightGBM] [Warning] lambda_l2 is set=1.8523641834042492, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.8523641834042492
[LightGBM] [Warning] lambda_l1 is set=4.994553000952651, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.994553000952651
[LightGBM] [Warning] bagging_fraction is set=0.9976539330187573, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9976539330187573
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] min_data_in_leaf is set=45, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=45
[LightGBM] [Warning] feature_fraction is set=0.7095710565305883, colsample_bytree=

In [47]:
#calculate train mae
train_mae=mean_absolute_error(y_pred_train,y_train)
test_mae=mean_absolute_error(y_pred_test,y_test)
print('train_mae',train_mae)
print('test_mae',test_mae)

train_mae 2.7677913753743857
test_mae 3.0754730752295147


In [48]:
train_r2_score=r2_score(y_pred_train,y_train)
test_r2_score=r2_score(y_pred_test,y_test)
print('train_r2_score',train_r2_score)
print('test_r2_score',test_r2_score)

train_r2_score 0.8431256643393743
test_r2_score 0.8072274575695322


In [52]:

cv_scores = cross_val_score(model,
                            x_train_trans,
                            y_train,cv=3,
                            scoring="neg_mean_absolute_error",
                            n_jobs=-1)

In [53]:

cv_scores

array([-3.04407952, -3.12431371, -3.09693588])

In [54]:
#logging

with mlflow.start_run():

  #log model,params
  mlflow.set_tag('model','stacking_regressor')

  mlflow.log_params(stacking_reg.get_params())

  #log metrics

  mlflow.log_metric('train_me',train_mae)
  mlflow.log_metric('test_mse',test_mae)

  mlflow.log_metric('train_r2_score',train_r2_score)
  mlflow.log_metric('test_r2_score',test_r2_score)

  mlflow.log_metric('cv_score',-(cv_scores.mean()))

  #log model
  mlflow.sklearn.log_model(stacking_reg,'model')



2025/04/03 03:03:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run magnificent-robin-75 at: https://dagshub.com/amitkumar981/swiggy-delivery-time-prediction.mlflow/#/experiments/9/runs/ed93ec5ea55049daac2cce0767a22fe4
🧪 View experiment at: https://dagshub.com/amitkumar981/swiggy-delivery-time-prediction.mlflow/#/experiments/9
